In [2]:
%load_ext autoreload
%autoreload 2

# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np
import opentrons
from  opentrons.types import Location, Point

from opentrons_helper import * 

from opentrons import simulate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
'''
Protocol in a nutshell
'''

V_aspirate = 200e-6
V_flask = 15e-3
c_peptone = 10 # g/L
c_yeast = 10 # g/L
c_sach = 100 #g/L
c_sach_var = (0,1,2,5,10,20,50,100) # # g/L
# compute the solution in the 15ml bottle:
# ammount in flask in gramm
a_yeast = c_yeast*V_flask
a_peptone = c_peptone*V_flask
a_sach = c_sach*V_flask
print("Prepare yeast: ",c_yeast*V_flask, "g per 15ml bottle")
print("Prepare peptone: ",c_peptone*V_flask, "g per 15ml bottle")
print("Prepare sachharose: ",c_sach*V_flask, "g per 15ml bottle")
'''
final concentration for sachharose if we mix it with water:
    C=(a1+a2)/(V1+V2)
     =>
     C_fina l = (C_stock*V_sachh)/(V_sachh+V_water)
     V_aspirate = V_sachh+V_water
     =>
     V_water = V_aspirate*(C_final-C_stock)/C_stock
'''
#%%
print("Stock solution: (Yeast, Peptone, Sachharose:")
print("Yeast, | ,  Peptone,  | , Sachharose:")
print("[g/L]"," | ", "[g/L]"," | ", "[g/L]")
for c_peptone in (0,1,2,5,10,20,50,100):
    for c_i_sach in c_sach_var:
        #compute amount of water to end up with concentration for sachharose
        V_water = V_aspirate*(c_sach-c_i_sach)/c_sach
        V_sachh = V_aspirate-V_water
    #    print("V_yeast : ", V_aspirate*1e6, "V_peptone: ", V_aspirate*1e6, "V_sachh: ",V_sachh*1e6, "V_water: ", V_water*1e6, "µl")   
        print((V_aspirate*c_yeast)/V_aspirate," | ", V_aspirate*c_peptone/V_aspirate, " | ",  round((c_sach*V_sachh)/V_aspirate,2))
        

Prepare yeast:  0.15 g per 15ml bottle
Prepare peptone:  0.15 g per 15ml bottle
Prepare sachharose:  1.5 g per 15ml bottle
Stock solution: (Yeast, Peptone, Sachharose:
Yeast, | ,  Peptone,  | , Sachharose:
[g/L]  |  [g/L]  |  [g/L]
10.0  |  0.0  |  0.0
10.0  |  0.0  |  1.0
10.0  |  0.0  |  2.0
10.0  |  0.0  |  5.0
10.0  |  0.0  |  10.0
10.0  |  0.0  |  20.0
10.0  |  0.0  |  50.0
10.0  |  0.0  |  100.0
10.0  |  1.0  |  0.0
10.0  |  1.0  |  1.0
10.0  |  1.0  |  2.0
10.0  |  1.0  |  5.0
10.0  |  1.0  |  10.0
10.0  |  1.0  |  20.0
10.0  |  1.0  |  50.0
10.0  |  1.0  |  100.0
10.0  |  2.0  |  0.0
10.0  |  2.0  |  1.0
10.0  |  2.0  |  2.0
10.0  |  2.0  |  5.0
10.0  |  2.0  |  10.0
10.0  |  2.0  |  20.0
10.0  |  2.0  |  50.0
10.0  |  2.0  |  100.0
10.0  |  5.0  |  0.0
10.0  |  5.0  |  1.0
10.0  |  5.0  |  2.0
10.0  |  5.0  |  5.0
10.0  |  5.0  |  10.0
10.0  |  5.0  |  20.0
10.0  |  5.0  |  50.0
10.0  |  5.0  |  100.0
10.0  |  10.0  |  0.0
10.0  |  10.0  |  1.0
10.0  |  10.0  |  2.0
10.0  |  1

In [14]:
'''
Connect to the microscope 
'''

from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="21.3.2.3", port=5000, is_simulate=True)
#!pip3 install zeroconf
#!pip3 install requests


Microscope is in simulation mode


In [5]:
# Connect to the opentrons robot
protocol = simulate.get_protocol_api('2.10')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!

# setup the labware in the trays
plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')
#plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='9')
plate_reagents = protocol.load_labware('opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap', location='10')
plate_reagents_water = protocol.load_labware('opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap', location='7')


# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',location='11')

# sample plate on microscope
# offset of the well plate in mm


In [6]:
from opentrons.types import Location, Point

# set speed
#protocol.max_speeds['X'] = 200 # default = 400
#protocol.max_speeds['Y'] = 200 # default = 300

# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11

# wells with reagents
i_yeast = 0
i_peptone = 1
i_sugar = 2

# different Well
i_water = 0


# define timig
t_bsa = 15
t_primary = 60
t_secondary = 60
t_fluo = 30*0

# move it down since opentrons expects the reagents to be furthter away from the ground
offset_z_reagents = -65

# define locations 
minimum_z_height = 150 # minimum height for arcs driven by the robot

# compute the positions
position_trash =  (365,350, 100)

position_idle =  (100,100, 150)#

# Calibrate

In [15]:
'''
calibrate tiprack
'''
#protocol.home()
i_pipette = 1
#pitty, but moving the xy coordinates doesn't work
print(i_pipette)
offset_pipette_rack=(1,3,0)
tiprack.set_calibration(Point(*offset_pipette_rack))

i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)
print(type(i_pipette))

1
My pipette number: 1and type: <class 'int'>
<class 'int'>


In [16]:
#from opentrons_client import OpentronsClient 
#ot2 = OpentronsClient(host="21.3.2.4", port=31950)
#ot2.toggle_light(True)
#ipot2.pipette_home(location="right")
#pos = ot2.positions()

{'on': True}

In [17]:
'''
calibrate illumination poisiton
'''
microscope.home()
#protocol.home()
#position_idle =  (100,300, 150)#
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
well_to_well_steps= 9100
#microscope.home()
position_sample_light = (250, 99, 190)

# This offset represents the position of well 7!
offset_x = 4000 # 1290+0*9000
offset_y = -1000# -1400+4*9000
offset_z = microscope.position["z"]
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
microscope.set_laser_led(0,1)
microscope.move((offset_x,offset_y,offset_z),absolute=True)
autofocus_dz = 500
autofocus_Nz = 7
is_autofocus = False
#offset_z = microscope.autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
#microscope.set_laser_led(0,1)


microscope.move((offset_x,offset_y,offset_z),absolute=True)

Microscope is homing...
Please add a pipette first!
Moving to: (250, 99, 190)


In [18]:

microscope.set_laser_led(0,1)

In [19]:
'''
Calibrate Wellposition for sample coordinates (96well plate)
pipette should dip in the wells on the microscope

'''
#pipette.home()
offset_z = microscope.position["z"]
home_position_microscope=(0,0,offset_z)
microscope.move((home_position_microscope))

minimum_z_height = 155
offset_plate_sample=(-28,68,127)
plate_microscope.set_calibration(Point(*offset_plate_sample))

for i_pos in range(4):
        position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
        print(position_sample_i)
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)




[251.38  142.24  135.885]
Moving to: (251.38, 142.24, 135.885)
[251.38  133.24  135.885]
Moving to: (251.38, 133.24, 135.885)
[251.38  124.24  135.885]
Moving to: (251.38, 124.24000000000001, 135.885)
[251.38  115.24  135.885]
Moving to: (251.38, 115.24000000000001, 135.885)
Moving to: (251.38, 115.24000000000001, 145.885)


In [20]:
# goes into all 3 reagent slots 
'''
calibrate reagetn slots
'''
pipette.home()

offset_x_reagents = -4
offset_y_reagents = 8
offset_z_reagents = -50 # smaller number means: go up
for i_pos in range(3):
    i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)
    position_reagent_i = np.array(plate_reagents.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)

# move out of the well again
move_to_coord(pipette, position_reagent_i, offset=(0,0,20), minimum_z_height=minimum_z_height)


TypeError: 'int' object is not subscriptable

# Start the experiment

In [21]:
# initialize strings
from datetime import datetime
timestamp = datetime.now().strftime("%Y_%m_%d-%I-%M-%S_%p")
i_experiment = 0



'''
##############################################################################
##########################HOW MANY WELLS DO WE WANT TO DISPENSE?!##########################
##############################################################################
'''


well_id_list = np.arange(0,8*10) # we don't want to go to the very end.

## Add Yeast solution ( 0.15g/15mL) -> 200 uL per pipette to the sample plate

In [ ]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

V_aspirate_yeast = 200 #uL 

for i_pos in range(len(well_id_list)):
    # get yeast 
    position_reagent_i = np.array(plate_reagents.wells()[i_yeast].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate_yeast)
    
    # add yeast to sample
    position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    time.sleep(1)
        
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



My pipette number: 2and type: <class 'int'>
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 142.24, 135.885)
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 133.24, 135.885)
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 124.24000000000001, 135.885)
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 115.24000000000001, 135.885)
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 106.24000000000001, 135.885)
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 97.24, 135.885)
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 88.24, 135.885)
Moving to: (14.21, 354.93, 10.949999999999996)
Moving to: (251.38, 79.24, 135.885)


##  Add peptone solution ( 0.015g/15mL) -> 200 uL per pipette to the sample plate

PERHAPS WE CAN ALSO MAKE THIS HERE "ADAPTIVE"? SO THAT YOU ENTER THE EXPERIMENT ID AND THE VALUE WILL AUTOMATICALLY BE EXPORTED/STORED? I KNOW THAT YOU WILL PREPARE THE BOTTLE WITH THE PEPTONE BEFOREHAND


```py
V_aspirate_peptone = 200 #uL
```


In [ ]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

V_aspirate_peptone = 200 #uL

for i_pos in range(len(well_id_list)):
    # get yeast 
    position_reagent_i = np.array(plate_reagents.wells()[i_peptone].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate_peptone)
    
    # add yeast to sample
    position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    time.sleep(1)
        
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



In [ ]:
#V_aspirate_water + V_aspirate_sugar = 200 



## Add sugar solution 1 ( 1.5g/15mL) plus water -> variable uL per pipette to the sample plate

In [ ]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

#V_aspirate_sugar = np.arange(len(well_id_list))

# sugar solution 1 to be added to wells from well plate 0 till 1/2 way through 
# water to be added in same range as above .
# j varies from 20 to 200 uL for 10 equally spaced values.


for i_pos in range(0,len(well_id_list//2)) and for j in np.linspace(20,200,10):
    V_aspirate_sugar = j
    V_aspirate_water = 200 - j
        # get sugar
    position_reagent_i = np.array(plate_reagents.wells()[i_sugar].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate_sugar[i_pos])

        # add sugar to sample 
    position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    time.sleep(1)
    
    
    #get water
    position_reagent_i = np.array(plate_reagents_water.wells()[i_water].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate_water)
    
    # add water to sample 
    position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    time.sleep(1)
        
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



## Add sugar solution 2 ( 0.015g/15mL) plus water-> variable uL per pipette to the sample plate


In [ ]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

#V_aspirate_sugar = np.arange(len(well_id_list))


# sugar solution 2 to be added to wells from 1/2 way through till end 
# water to be added in same range as above .
# j varies from 20 to 200 uL for 10 equally spaced values.
for i_pos in range(len(well_id_list//2),len(well_id_list)) and for j in np.linspace(20,200,10): :
    V_aspirate_sugar = j
    V_aspirate_water = 200 - j
        # get sugar
    position_reagent_i = np.array(plate_reagents.wells()[i_sugar].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate_sugar[i_pos])

        # add sugar to sample 
    position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    time.sleep(1)
    
    
    #get water
    position_reagent_i = np.array(plate_reagents_water.wells()[i_water].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate_water)
    
    # add water to sample 
    position_sample_i = np.array(plate_microscope.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    time.sleep(1)
        
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



## 12 hour time lapse

In [ ]:
# overnight ( 6 hour time lapse)
time.sleep(60*60*12)

## Microscope position config

In [ ]:
# lets assume the robots light source is in place 
microscope.set_laser_led(i_laser=0, i_led=1)


# lets define the position for the seventh well (ID)
offset_x = 5000
offset_y = -1000
offset_z = 1500

microscope.move((offset_x, offset_y, offset_z))

print(microscope.position)

offset_z
I_laser = 0
I_led = 1
id = 0



## Take focus-defocus images after time-lapse 

In [ ]:
# create the object for the well scanner 
wellscanner = Hi2Module( microscope, well_to_well_steps=9000,
             N_wells = 96, Nx = 8, Ny = 12)

# set the well-position of the well number  7 
#wellscanner.set_offset_for_well_id_7(offset_x, offset_y)


microscope.home()
for id in range(0,95):

    # first go to well ID 
    wellscanner.move_to_well_id(offset_z=None, well_index=id)
    
    # get the acutal position of the microscope
    current_x, current_y, _ = microscope.position['x'],  microscope.position['y'],  microscope.position['z']
    
   # current_x += np.radnom.rnadniintcurrent_x+ 500
    
    microscope.set_laser_led(I_laser,I_led)
    
    offset_z = microscope.autofocus_coarse(dz=1000, nz=15)
    current_x, current_y, _ = microscope.position['x'],  microscope.position['y'],  microscope.position['z']
    

    # 1. capture the in-focus image
    base_filename = 'data_generation_GAN_17Jun_'
    params = {
        "filename": base_filename+"de_focus_"+str(id),
    }
    microscope.set_laser_led(I_laser,I_led)
    microscope.move((offset_x, offset_y, offset_z))
    time.sleep(1)
    microscope.capture_image_to_disk(params)

    max_dz = 200 # the maximum value the focus may differ => plus minus max_dz/2
    offset_z_defocus = offset_z+ np.random.randint(max_dz)-(max_dz//2)
    print(offset_z_defocus)

    # 2. cpature the defocus image 
    params = {
        "filename": base_filename+"in_focus_"+str(id),
    }

    microscope.move((current_x, current_y, offset_z_defocus))
    time.sleep(1) # debounce any vibvration
    microscope.capture_image_to_disk(params)



### Additional steps : 

In [ ]:
# Do we need this ?

#microscope.plate_shaking(1)
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
mytime = -time.time()

t_period = 20 # min - how often should the expimrent be carried out?
is_autofocus = True
is_find_prefocus = True
autofocus_dz = 1500
autofocus_Nz = 15
#i_experiment = 0
focus_pos_list = []
while True:

    if time.time()-mytime > (t_period*60):
        mytime = time.time()
        i_experiment += 1
    
        # perform a testing well scan by moving robot to light position and do a whole plate scan
        #move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
        wellscanner.wellscan_list(
                    i_experiment,
                    well_id_list,
                    autofocus_dz, 
                    autofocus_Nz,
                    name_experiment="wellscan_yeast__new_",
                    is_autofocus=is_autofocus, 
                    I_laser=0, 
                    I_led=1,
                    t_debounce=.5,
                    is_find_prefocus=is_find_prefocus,
                    process_func=None)
        is_autofocus = False
        is_find_prefocus = False
        
    else:
        time.sleep(1)
        

    



            
        

In [ ]:
microscope.set_laser_led(0,1)
wellscanner.move_to_well_id(7)